In [16]:
import time 
import scrapy
import requests

In [15]:
max_institute_id = 423
min_institute_id = 357

In [17]:
import re
import string

def clean_data(raw_data):
    cleaned_data = []

    for item in raw_data:
        item[-1] = item[-1].translate(str.maketrans("", "", string.punctuation))

        cleaned_data.append(item)

    return cleaned_data



In [13]:
valid_institute_ids = []

In [18]:
total_course_codes = []
base_url = "https://c2c.ontransfer.ca/ajax_en.php?arid=c2c_load_autocomplete_array" 
for institute_id in range(min_institute_id, max_institute_id+1):
    response = requests.get(base_url + "&institution_id=" + str(institute_id))
    if response.status_code == 200:
        course_codes = response.text.split("|*|")
        if not course_codes:
            continue
        valid_institute_ids.append(institute_id)
        course_codes = [course_code.strip().split("|**|") for course_code in course_codes if course_code]
        total_course_codes.extend(course_codes)
print(f"Total course_codes: {len(total_course_codes)}")
file = '.valid_id.txt'

with open(file, 'w') as f:
    for valid_institute_id in valid_institute_ids:
        f.write(str(valid_institute_id) + '\n')

Total course_codes: 196012


In [9]:
from openpyxl import Workbook

course_code_workbook = Workbook()
course_code_sheet = course_code_workbook.active
course_code_sheet.append(['Prefix', 'Number', 'Suffix', 'Title'])
for valid_course_code in total_course_codes:
    course_code_sheet.append(valid_course_code)
    # print(valid_course_code)
    # break
course_code_workbook.save('course_codes.xlsx')
    

In [11]:
print(len(valid_institute_ids)*len(total_course_codes))

13132804


In [2]:
form_data = {
    'page':'c2c',
    'user_action':'search_t2',
    'C2CTargetInstitution':'398',
    'C2CTargetPrefix':'AA',
    'C2CTargetNumber':'3035',
    'C2CTargetSuffix':'',
    'C2CTargetTitle':'Understanding the Family',
    'back_link':'page=c2c&user_action=search_t2',
    'results':'Find Equivalencies'
}

In [25]:
# !pip install pandas
from openpyxl import Workbook
import pandas as pd
url_workbook = Workbook()
url_sheet = url_workbook.active
url_sheet.append(['URLS'])

base_url = 'https://c2c.ontransfer.ca/index_en.php'
cnt = 0
df = pd.read_excel('course_codes.xlsx')
for prefix in df.values:
    try:
        pref, num, suff, title = prefix
        for institute_id in valid_institute_ids:
            URL = f'https://c2c.ontransfer.ca/index_en.php?page=c2c&user_action=search_t2&C2CTargetInstitution={institute_id}&C2CTargetPrefix={pref}&C2CTargetNumber={num}&C2CTargetSuffix={suff}&C2CTargetTitle={title}&back_link=page&results=Find+Equivalencies'
            # url_sheet.append([f'=HYPERLINK("{URL}", "Link")'])
            cell = url_sheet.cell(row=len(url_sheet['A']) + 1, column=1)
            cell.value = URL
            cell.hyperlink = URL
        cnt += 1
        if cnt > 100:
            break
    except Exception as e:
        print(e)
        pass
url_workbook.save('urls.xlsx')

In [47]:
# Scrapy
import scrapy
class TabularSpider(scrapy.Spider):
    name = 'tabular'
    start_urls = ['https://c2c.ontransfer.ca/index_en.php?page=c2c&user_action=search_t2&C2CTargetInstitution=366&back_link=page%3Dc2c%26user_action%3Dsearch_t2&results=Find+Equivalencies']
    def parse(self, response):
        if response.status == 200:
            table = response.xpath('//table')
            rows = table.xpath('.//tr')
            for row in rows:
                cells = row.xpath('.//td')
                for cell in cells:
                    print(cell.xpath('.//text()').extract())    


In [61]:
table = driver.find_element(By.XPATH, '//*[@id="form_c2c_search_results"]/fieldset/table')

In [62]:
thead = table.find_element(By.TAG_NAME, "thead")

In [82]:
header_rows = thead.find_elements(By.TAG_NAME, "tr")

In [86]:
tbody = table.find_element(By.TAG_NAME, 'tbody')
body_rows = tbody.find_elements(By.TAG_NAME, 'tr')

In [83]:
len(header_rows)

2

In [94]:
workbook = Workbook()
worksheet = workbook.active

In [99]:
all_rows = body_rows + header_rows
for row in all_rows:
    try:
        cols = row.find_elements(By.TAG_NAME, 'th')
        cols_data = [col.text for col in cols]
        print(cols_data)
    except:
        cols = row.find_elements(By.TAG_NAME, 'td')
        cols_data = [col.text for col in cols][1:]
        print(cols_data)
    worksheet.append(cols_data)
workbook.save('table_data.xlsx')

['', 'McMaster University', 'ABST1P01 - INTRODUCTION TO MOHAWK I', 'C', 'Brock University', '', 'ABST1P01 - INTRODUCTION TO MOHAWK I', ' ']
['', 'Trent University', 'NATST 205 - INTRODUCTION TO MOHAWK I', 'C', 'Brock University', '', 'ABST1P01 - INTRODUCTION TO MOHAWK I', ' ']
['', 'Trent University', 'NS225 - INTRODUCTION TO MOHAWK I', 'C', 'Brock University', '', 'ABST1P01 - INTRODUCTION TO MOHAWK I', ' ']
[' ', ' ']
[' ', ' ']


3

In [42]:
wb.close()